# Validation Deep Dive

This notebook analyzes **real production results** from our NBA player props system:
- **Win rate by edge tier** - How well does model confidence correlate with outcomes?
- **Calibration analysis** - Are probability estimates reliable?
- **ROI over time** - How consistent is performance?
- **Market-by-market breakdown** - Which markets perform best?

**Data Source**: Real prediction files from `predictions/` validated against PostgreSQL database

In [ ]:
import json
import os
from pathlib import Path
from datetime import datetime, timedelta
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Database connection
import psycopg2

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

# Color scheme
COLORS = {
    'win': '#2ecc71',
    'loss': '#e74c3c',
    'points': '#3498db',
    'rebounds': '#9b59b6',
    'assists': '#f39c12'
}

## 1. Database Connection

Connect to PostgreSQL to fetch actual game results for validation.

In [ ]:
DB_CONFIG = {
    "host": "localhost",
    "port": 5536,
    "user": "nba_user",
    "password": os.getenv("DB_PASSWORD"),
    "database": "nba_players",
}

def get_db_connection():
    """Get database connection."""
    return psycopg2.connect(**DB_CONFIG)

# Test connection
try:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM player_game_logs WHERE game_date >= '2026-01-01'")
    count = cursor.fetchone()[0]
    print(f"Database connected. {count} game logs available for 2026.")
    cursor.close()
    conn.close()
except Exception as e:
    print(f"Database connection failed: {e}")
    print("Ensure Docker containers are running: cd docker && docker-compose up -d")

## 2. Load Real Prediction Files

In [ ]:
PREDICTIONS_DIR = Path("../nba/betting_xl/predictions")

def load_all_predictions(start_date: str, end_date: str) -> pd.DataFrame:
    """
    Load all prediction files within date range.
    Returns DataFrame with all picks.
    """
    start = datetime.strptime(start_date, "%Y-%m-%d").date()
    end = datetime.strptime(end_date, "%Y-%m-%d").date()
    
    all_picks = []
    
    current = start
    while current <= end:
        date_str = current.strftime("%Y-%m-%d")
        date_compact = date_str.replace("-", "")
        
        # Try different file naming conventions
        files_to_try = [
            PREDICTIONS_DIR / f"xl_picks_{date_str}.json",
            PREDICTIONS_DIR / f"xl_picks_{date_compact}.json",
            PREDICTIONS_DIR / f"pro_picks_{date_str}.json",
            PREDICTIONS_DIR / f"pro_picks_{date_compact}.json",
        ]
        
        for filepath in files_to_try:
            if filepath.exists():
                try:
                    with open(filepath) as f:
                        data = json.load(f)
                        for pick in data.get("picks", []):
                            pick["game_date"] = date_str
                            pick["source_file"] = filepath.name
                            all_picks.append(pick)
                except json.JSONDecodeError:
                    continue
        
        current += timedelta(days=1)
    
    return pd.DataFrame(all_picks)

# Load predictions from January 2026
START_DATE = "2026-01-01"
END_DATE = "2026-01-28"

picks_df = load_all_predictions(START_DATE, END_DATE)
print(f"Loaded {len(picks_df)} picks from {START_DATE} to {END_DATE}")
print(f"\nFiles found: {picks_df['source_file'].nunique()}")
print(f"Date range: {picks_df['game_date'].min()} to {picks_df['game_date'].max()}")

In [ ]:
# Display sample picks
if len(picks_df) > 0:
    display_cols = ['player_name', 'stat_type', 'side', 'line', 'probability', 'game_date']
    available_cols = [c for c in display_cols if c in picks_df.columns]
    picks_df[available_cols].head(10)

## 3. Fetch Actual Results from Database

In [ ]:
def normalize_name(name: str) -> str:
    """Normalize player name for matching."""
    suffixes = [" Jr", " Jr.", " III", " II", " IV", " Sr", " Sr."]
    normalized = name.strip()
    for suffix in suffixes:
        if normalized.endswith(suffix):
            normalized = normalized[:-len(suffix)].strip()
    return normalized.lower()

def get_actual_results(dates: list) -> dict:
    """
    Fetch actual game results for given dates from database.
    Returns dict: (player_name, date) -> {stat_type: actual_value}
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    query = """
        SELECT
            p.full_name,
            l.game_date,
            l.points,
            l.rebounds,
            l.assists,
            l.three_pointers_made
        FROM player_game_logs l
        JOIN player_profile p ON l.player_id = p.player_id
        WHERE l.game_date = ANY(%s)
    """
    
    cursor.execute(query, (dates,))
    
    results = {}
    for row in cursor.fetchall():
        name, game_date, points, rebounds, assists, threes = row
        date_str = game_date.strftime("%Y-%m-%d") if hasattr(game_date, 'strftime') else str(game_date)
        
        stats = {
            "POINTS": points,
            "REBOUNDS": rebounds, 
            "ASSISTS": assists,
            "THREES": threes
        }
        
        # Store with both original and normalized names
        results[(name, date_str)] = stats
        results[(normalize_name(name), date_str)] = stats
    
    cursor.close()
    conn.close()
    
    return results

# Fetch actuals for all dates in our picks
if len(picks_df) > 0:
    unique_dates = picks_df['game_date'].unique().tolist()
    actuals = get_actual_results(unique_dates)
    print(f"Fetched {len(actuals) // 2} player game logs from database")

## 4. Validate Predictions Against Actuals

In [ ]:
def validate_pick(pick: dict, actuals: dict) -> dict:
    """
    Validate a single pick against actual results.
    Returns validation result with outcome and profit.
    """
    player = pick["player_name"]
    stat_type = pick["stat_type"]
    date = pick["game_date"]
    
    # Handle different line field names
    line = pick.get("best_line") or pick.get("line")
    side = pick.get("side", "OVER")
    
    # Try to find actual result
    actual = None
    
    # Try exact match
    key = (player, date)
    if key in actuals:
        actual = actuals[key].get(stat_type)
    
    # Try normalized name
    if actual is None:
        key = (normalize_name(player), date)
        if key in actuals:
            actual = actuals[key].get(stat_type)
    
    if actual is None:
        return {"status": "NO_DATA", "reason": "No game log found"}
    
    # Determine outcome
    if side == "OVER":
        won = actual > line
        push = actual == line
    else:
        won = actual < line
        push = actual == line
    
    # Calculate profit (-110 odds)
    if push:
        profit = 0.0
        outcome = "PUSH"
    elif won:
        profit = 0.909
        outcome = "WIN"
    else:
        profit = -1.0
        outcome = "LOSS"
    
    return {
        "status": "VALIDATED",
        "outcome": outcome,
        "actual": actual,
        "line": line,
        "diff": actual - line,
        "profit": profit,
        "won": won and not push
    }

# Validate all picks
validated_picks = []

for _, pick in picks_df.iterrows():
    result = validate_pick(pick.to_dict(), actuals)
    if result["status"] == "VALIDATED":
        validated_picks.append({
            "player_name": pick["player_name"],
            "stat_type": pick["stat_type"],
            "game_date": pick["game_date"],
            "line": result["line"],
            "actual": result["actual"],
            "outcome": result["outcome"],
            "profit": result["profit"],
            "won": result["won"],
            "p_over": pick.get("probability", pick.get("p_over", 0.5)),
            "edge": pick.get("projection_diff", pick.get("edge", 0)),
        })

validation_df = pd.DataFrame(validated_picks)
print(f"\nValidated {len(validation_df)} picks against database")
print(f"Win Rate: {validation_df['won'].mean():.1%}")
print(f"Total Profit: {validation_df['profit'].sum():+.2f} units")

## 5. Win Rate by Edge Tier

In [ ]:
if len(validation_df) > 0 and 'edge' in validation_df.columns:
    # Create edge tiers based on actual data distribution
    validation_df['edge_abs'] = validation_df['edge'].abs()
    
    validation_df['edge_tier'] = pd.cut(
        validation_df['edge_abs'],
        bins=[0, 0.5, 1.0, 2.0, 3.0, 100],
        labels=['0-0.5', '0.5-1', '1-2', '2-3', '3+']
    )
    
    # Calculate stats by tier
    tier_stats = validation_df.groupby('edge_tier').agg({
        'won': ['sum', 'count', 'mean'],
        'profit': 'sum'
    }).round(3)
    
    tier_stats.columns = ['wins', 'bets', 'win_rate', 'profit']
    tier_stats['roi'] = (tier_stats['profit'] / tier_stats['bets'] * 100).round(1)
    tier_stats['win_rate_pct'] = (tier_stats['win_rate'] * 100).round(1)
    
    print("Win Rate by Edge Tier (Real Data)")
    print("="*50)
    display(tier_stats[['wins', 'bets', 'win_rate_pct', 'profit', 'roi']])

In [ ]:
if len(validation_df) > 0 and 'edge_tier' in validation_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Win rate by tier
    tier_data = tier_stats.dropna()
    colors = ['#e74c3c' if wr < 52.4 else '#2ecc71' for wr in tier_data['win_rate_pct']]
    bars = axes[0].bar(tier_data.index.astype(str), tier_data['win_rate_pct'], color=colors, edgecolor='black')
    axes[0].axhline(y=52.4, color='red', linestyle='--', label='Breakeven (52.4%)')
    axes[0].set_xlabel('Edge Tier')
    axes[0].set_ylabel('Win Rate (%)')
    axes[0].set_title('Win Rate by Edge Tier (Real Data)')
    axes[0].legend()
    
    for bar, val in zip(bars, tier_data['win_rate_pct']):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                     f'{val:.1f}%', ha='center', va='bottom', fontsize=10)
    
    # ROI by tier
    roi_colors = ['#e74c3c' if r < 0 else '#2ecc71' for r in tier_data['roi']]
    bars = axes[1].bar(tier_data.index.astype(str), tier_data['roi'], color=roi_colors, edgecolor='black')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_xlabel('Edge Tier')
    axes[1].set_ylabel('ROI (%)')
    axes[1].set_title('ROI by Edge Tier (Real Data)')
    
    for bar, val in zip(bars, tier_data['roi']):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                     f'{val:+.1f}%', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.savefig('edge_tier_performance.png', dpi=150, bbox_inches='tight')
    plt.show()

## 6. ROI Over Time (Cumulative)

In [ ]:
if len(validation_df) > 0:
    # Calculate daily and cumulative stats
    daily_stats = validation_df.groupby('game_date').agg({
        'won': ['sum', 'count'],
        'profit': 'sum'
    })
    daily_stats.columns = ['wins', 'bets', 'daily_profit']
    daily_stats['win_rate'] = daily_stats['wins'] / daily_stats['bets']
    daily_stats['cumulative_profit'] = daily_stats['daily_profit'].cumsum()
    daily_stats['cumulative_bets'] = daily_stats['bets'].cumsum()
    daily_stats['cumulative_roi'] = daily_stats['cumulative_profit'] / daily_stats['cumulative_bets'] * 100
    
    daily_stats.head(10)

In [ ]:
if len(validation_df) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Cumulative profit
    ax1 = axes[0]
    dates = range(len(daily_stats))
    ax1.fill_between(dates, 0, daily_stats['cumulative_profit'], 
                     where=daily_stats['cumulative_profit'] >= 0, 
                     color='#2ecc71', alpha=0.3)
    ax1.fill_between(dates, 0, daily_stats['cumulative_profit'], 
                     where=daily_stats['cumulative_profit'] < 0, 
                     color='#e74c3c', alpha=0.3)
    ax1.plot(dates, daily_stats['cumulative_profit'], 'b-', linewidth=2, label='Cumulative Profit')
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.set_ylabel('Cumulative Profit (units)', fontsize=12)
    ax1.set_title('Cumulative Profit Over Time (Real Data)', fontsize=14)
    ax1.legend()
    ax1.set_xticks(dates[::3] if len(dates) > 10 else dates)
    ax1.set_xticklabels([d for d in daily_stats.index[::3] if len(dates) > 10 else daily_stats.index], rotation=45)
    
    # Daily win rate
    ax2 = axes[1]
    colors = ['#2ecc71' if wr >= 0.524 else '#e74c3c' for wr in daily_stats['win_rate']]
    ax2.bar(dates, daily_stats['win_rate'] * 100, color=colors, alpha=0.7, edgecolor='black')
    ax2.axhline(y=52.4, color='red', linestyle='--', label='Breakeven', linewidth=2)
    ax2.axhline(y=60, color='green', linestyle=':', alpha=0.7, label='Target (60%)')
    ax2.set_ylabel('Daily Win Rate (%)', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_title('Daily Win Rate (Real Data)', fontsize=14)
    ax2.legend()
    ax2.set_ylim(0, 100)
    ax2.set_xticks(dates[::3] if len(dates) > 10 else dates)
    ax2.set_xticklabels([d for d in daily_stats.index[::3] if len(dates) > 10 else daily_stats.index], rotation=45)
    
    plt.tight_layout()
    plt.savefig('roi_over_time.png', dpi=150, bbox_inches='tight')
    plt.show()

## 7. Market-by-Market Breakdown

In [ ]:
if len(validation_df) > 0:
    market_stats = validation_df.groupby('stat_type').agg({
        'won': ['sum', 'count', 'mean'],
        'profit': 'sum'
    }).round(3)
    
    market_stats.columns = ['wins', 'bets', 'win_rate', 'profit']
    market_stats['roi'] = (market_stats['profit'] / market_stats['bets'] * 100).round(1)
    market_stats['win_rate_pct'] = (market_stats['win_rate'] * 100).round(1)
    
    print("Performance by Market (Real Data)")
    print("="*60)
    display(market_stats[['wins', 'bets', 'win_rate_pct', 'profit', 'roi']])

In [ ]:
if len(validation_df) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    markets = market_stats.index.tolist()
    colors = [COLORS.get(m.lower(), 'gray') for m in markets]
    
    # Win rate comparison
    axes[0].bar(markets, market_stats['win_rate_pct'], color=colors, edgecolor='black')
    axes[0].axhline(y=52.4, color='red', linestyle='--', label='Breakeven')
    axes[0].set_ylabel('Win Rate (%)')
    axes[0].set_title('Win Rate by Market (Real Data)')
    axes[0].legend()
    for i, (m, v) in enumerate(zip(markets, market_stats['win_rate_pct'])):
        axes[0].text(i, v + 1, f'{v:.1f}%', ha='center', fontsize=11)
    
    # ROI comparison
    roi_colors = ['#2ecc71' if r > 0 else '#e74c3c' for r in market_stats['roi']]
    axes[1].bar(markets, market_stats['roi'], color=roi_colors, edgecolor='black')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_ylabel('ROI (%)')
    axes[1].set_title('ROI by Market (Real Data)')
    for i, (m, v) in enumerate(zip(markets, market_stats['roi'])):
        axes[1].text(i, v + 1, f'{v:+.1f}%', ha='center', fontsize=11)
    
    # Bet volume
    axes[2].bar(markets, market_stats['bets'], color=colors, edgecolor='black')
    axes[2].set_ylabel('Number of Bets')
    axes[2].set_title('Bet Volume by Market (Real Data)')
    for i, (m, v) in enumerate(zip(markets, market_stats['bets'])):
        axes[2].text(i, v + 2, f'{int(v)}', ha='center', fontsize=11)
    
    plt.tight_layout()
    plt.savefig('market_breakdown.png', dpi=150, bbox_inches='tight')
    plt.show()

## 8. Summary Statistics

In [ ]:
if len(validation_df) > 0:
    print("="*60)
    print(f"VALIDATION SUMMARY - {START_DATE} to {END_DATE}")
    print("="*60)
    print(f"\nTotal Bets:     {len(validation_df)}")
    print(f"Wins:           {int(validation_df['won'].sum())} ({validation_df['won'].mean()*100:.1f}%)")
    print(f"Losses:         {int((~validation_df['won']).sum())}")
    print(f"Total Profit:   {validation_df['profit'].sum():+.2f} units")
    print(f"ROI:            {validation_df['profit'].sum() / len(validation_df) * 100:+.1f}%")
    
    print("\n" + "-"*60)
    print("BY MARKET")
    print("-"*60)
    for market in market_stats.index:
        row = market_stats.loc[market]
        losses = int(row['bets'] - row['wins'])
        print(f"{market:10s}: {int(row['wins']):3d}W / {losses:3d}L = {row['win_rate_pct']:.1f}% WR | ROI: {row['roi']:+.1f}%")
    
    print("\n" + "-"*60)
    print("DATA SOURCE")
    print("-"*60)
    print(f"Prediction files: {PREDICTIONS_DIR}")
    print(f"Database: {DB_CONFIG['database']} @ {DB_CONFIG['host']}:{DB_CONFIG['port']}")
    print("\nAll data is REAL - loaded from production prediction files and validated against PostgreSQL database.")